## Project 4: West Nile Virus Detection



### Problem Statement

To analyse weather data and GIS data to build a statistical model for the **optimal prediction of the presence, time, location and species of West Nile Virus (WNV) in Chicago.**

Concurrently, The Chicago Municipality and Chicago Department of Public Health is interested in identifying the highest number of WNV cases possible (sensitivity) rather than avoid False Positives, (where WNV is predicted but there isn't a case). 

This is due the high cost of an undetected WNV (due to health implications) as compared to the cost involved in spraying an area with presticide due to a false alarm.

Success is evaluated by ensuring that the model has the **highest Area Under the Curve score**, which signifies that the model is robust enough to allow flexible choice of thresholds. This can be verified on the Kaggle Website.

The Chicago Municipality and Chicago Department of Public Health can choose the model with the **highest sensitivity when WNV is pandemic** and subsequently, model with the **highest precision** when the number of incidences is lower.

### Executive Summary

- West Nile Virus is most commmonly spread through infected mosquitos
- 20% of people who are infected, develop symptoms ranging from fever, to serious neurological illness and death.
- In 2002, first human cases of WNV	were reported in Chicago. By 2004 the City of Chicago and the Chicago Department of	Public Health (CDPH) had established a comprehensive surveillance and control program.
- Every week, mosquitos in traps across the city are tested for the virus. The results of these tests infludence when and where the city will spray pesticides.
- Given weather, location, testing, and spraying data, Chicago Municipality and CDPH 






### Contents:
- [Train/Test Data Import](#Train/Test-Data-Import)
- [Explatory Data Analysis and Cleaning](#Explatory-Data-Analysis-and-Cleaning)
- [Feature Engineering](#Feature-Engineering)
- [Assessment of P values, and Modelling](#Assessment-of-P-values,-and-Modelling)


In [150]:
#*All libraries used should be added here*
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

## Train/Test Data Import 

#### Read in Train and Test Data


In [151]:
trainDf = pd.read_csv('train.csv')
trainDf.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


Since the lat/long are derived attributes based on address, block, street, etc., we can drop these columns and only use lat/long to determine geo-location.

In [152]:
trainDf.drop(['Address','Block','Street','AddressNumberAndStreet'],axis=1,inplace=True)
trainDf.head()

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,9,1,0
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,9,1,0
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,9,1,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,8,1,0
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,8,4,0


In [153]:
trainDf.shape

(10506, 8)

The train dataset contains 10506 rows and 12 columns.

In [154]:
trainDf.isnull().sum()

Date               0
Species            0
Trap               0
Latitude           0
Longitude          0
AddressAccuracy    0
NumMosquitos       0
WnvPresent         0
dtype: int64

There are no null values in the train dataset, which simplifies the data cleaning process.

In [155]:
trainDf.describe()

,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,41.841139,-87.699908,7.819532,12.853512,0.052446
std,0.112742,0.096514,1.452921,16.133816,0.222936
min,41.644612,-87.930995,3.000000,1.000000,0.000000
25%,41.732984,-87.760070,8.000000,2.000000,0.000000
50%,41.846283,-87.694991,8.000000,5.000000,0.000000
75%,41.954690,-87.627796,9.000000,17.000000,0.000000
max,42.017430,-87.531635,9.000000,50.000000,1.000000


Focus on the 'WnvPresent' column. Based on the information above, the dataset we have is highly inbalanced (only 5% of the rows contain data points which represents the presence of the WNV virus). This would be a problem for our model subsequently; hence, some additional effort would be required to balance out the dataset (e.g. by bootstrapping, oversampling, undersampling, SMOTE)

In [156]:
trainDf.columns

Index(['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent'],
      dtype='object')

In [157]:
trainDf.dtypes

Date                object
Species             object
Trap                object
Latitude           float64
Longitude          float64
AddressAccuracy      int64
NumMosquitos         int64
WnvPresent           int64
dtype: object

Convert the date column to datetime object so that subsequent operations can be done.

In [158]:
trainDf['Date'] = pd.to_datetime(trainDf['Date'])

In [159]:
trainDf.dtypes

Date               datetime64[ns]
Species                    object
Trap                       object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object

Import spray.csv file which may provide additional insights to the data.

In [160]:
sprayDf = pd.read_csv('spray.csv')
sprayDf.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [161]:
sprayDf.shape

(14835, 4)

The spray dataset contains 14835 rows and 4 columns.

In [162]:
sprayDf.isnull().sum()

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64

There are some null values in the 'Time' column, but not in the 'Date' column. There is no need to drop any rows since we can just make use of 'Date' for subsequent analysis.

In [163]:
sprayDf.dtypes

Date          object
Time          object
Latitude     float64
Longitude    float64
dtype: object

Similar to above, convert 'Date' column to datetime object for operations.

In [164]:
sprayDf['Date'] = pd.to_datetime(sprayDf['Date'])

In [165]:
sprayDf.dtypes

Date         datetime64[ns]
Time                 object
Latitude            float64
Longitude           float64
dtype: object

Merge trainDf and sprayDf datasets into a single dataset, using the geo-location as match key. Since geo-location has high resolution, we round the lat/long values in both dataframes to 3dp (~= 100 metres). 

In [166]:
trainDf['Latitude'] = trainDf['Latitude'].round(3)
trainDf['Longitude'] = trainDf['Longitude'].round(3)

In [167]:
sprayDf['Latitude'] = sprayDf['Latitude'].round(3)
sprayDf['Longitude'] = sprayDf['Longitude'].round(3)

Aggregate the spraying data points and calculate the count, max, min, peak-to-peak values for each lat/long geo-location.

In [168]:
spray1 = sprayDf.groupby(['Latitude','Longitude'])['Date'].agg(['count','max','min']).reset_index()
spray1['ptp'] = spray1['max'] - spray1['min']
spray1['ptp'] = spray1['ptp'].dt.days
spray1 = spray1.sort_values(by='ptp',ascending=False)
spray1.head()

,Latitude,Longitude,count,max,min,ptp
5908,41.997,-87.811,3,2013-09-05,2011-09-07,729
4374,41.954,-87.784,4,2013-08-16,2013-08-15,1
2527,41.911,-87.750,2,2013-08-16,2013-08-15,1
4454,41.957,-87.784,3,2013-08-16,2013-08-15,1
4704,41.965,-87.782,5,2013-08-16,2013-08-15,1


In [169]:
spray1.tail()

,Latitude,Longitude,count,max,min,ptp
2168,41.893,-87.768,3,2013-08-15,2013-08-15,0
2167,41.893,-87.770,2,2013-08-15,2013-08-15,0
2166,41.893,-87.772,1,2013-08-15,2013-08-15,0
2165,41.892,-87.704,1,2013-08-22,2013-08-22,0
6516,42.396,-88.096,2,2011-08-29,2011-08-29,0


In [170]:
spray1['ptp'].value_counts()

0      6511
1         5
729       1
Name: ptp, dtype: int64

In [171]:
spray1[spray1['ptp']>0]

,Latitude,Longitude,count,max,min,ptp
5908,41.997,-87.811,3,2013-09-05,2011-09-07,729
4374,41.954,-87.784,4,2013-08-16,2013-08-15,1
2527,41.911,-87.750,2,2013-08-16,2013-08-15,1
4454,41.957,-87.784,3,2013-08-16,2013-08-15,1
4704,41.965,-87.782,5,2013-08-16,2013-08-15,1
4703,41.965,-87.783,3,2013-08-16,2013-08-15,1


In [231]:
spray1.to_csv('spray1.csv',index=False)
sprayDf.to_csv('sprayDf.csv',index=False)

For most geo-locations, there is only 1 spray date. For those with ptp>1, we will need to examine which dates to use for the subsequent computations.

Do a left merge on the trainDf dataframe. The resulting combined dataframe will have the same number of rows as the trainDf dataframe.

In [172]:
sptraininner=pd.merge(trainDf,spray1,on=['Longitude','Latitude'],how='inner',indicator=True)
sptraininner.head()

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,count,max,min,ptp,_merge
0,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974,-87.825,8,1,0,1,2011-09-07,2011-09-07,0,both
1,2007-05-29,CULEX RESTUANS,T015,41.974,-87.825,8,4,0,1,2011-09-07,2011-09-07,0,both
2,2007-06-05,CULEX PIPIENS/RESTUANS,T015,41.974,-87.825,8,1,0,1,2011-09-07,2011-09-07,0,both
3,2007-06-05,CULEX RESTUANS,T015,41.974,-87.825,8,2,0,1,2011-09-07,2011-09-07,0,both
4,2007-06-26,CULEX RESTUANS,T015,41.974,-87.825,8,1,0,1,2011-09-07,2011-09-07,0,both


In [173]:
sptraininner.shape

(963, 13)

In [174]:
sptraininner[sptraininner['ptp']>0]

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,count,max,min,ptp,_merge


In [175]:
trainSprayCombined = pd.merge(trainDf,spray1,on=['Latitude','Longitude'],how='left')
trainSprayCombined.head()

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,count,max,min,ptp
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.955,-87.801,9,1,0,NaN,NaT,NaT,NaN
1,2007-05-29,CULEX RESTUANS,T002,41.955,-87.801,9,1,0,NaN,NaT,NaT,NaN
2,2007-05-29,CULEX RESTUANS,T007,41.995,-87.769,9,1,0,NaN,NaT,NaT,NaN
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974,-87.825,8,1,0,1.0,2011-09-07,2011-09-07,0.0
4,2007-05-29,CULEX RESTUANS,T015,41.974,-87.825,8,4,0,1.0,2011-09-07,2011-09-07,0.0


In [176]:
trainSprayCombined.shape

(10506, 12)

In [177]:
trainSprayCombined[trainSprayCombined['ptp']>0]

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,count,max,min,ptp


After merging, there are no more rows with ptp>0. This means that the 'max' and 'min' spray dates are the same for all data points in this combined dataframe.

In [178]:
trainSprayCombined['Date'] = pd.to_datetime(trainSprayCombined['Date'])

**Feature Engineering** to create new column 'Most_Recent_Spray'. This column represents the most recent spray (from sprayDf), in terms of number of days ago, with respect to the data collection date (from trainDf).

In [179]:
trainSprayCombined['Most_Recent_Spray'] = trainSprayCombined['Date']-trainSprayCombined['min']
trainSprayCombined['Most_Recent_Spray'] = trainSprayCombined['Most_Recent_Spray'].dt.days
trainSprayCombined['Most_Recent_Spray'] = trainSprayCombined['Most_Recent_Spray'].apply(lambda x: x if x > 0 else 3650)

Assumption: For rows with no spray record (NaN) and rows with negative recent spray days (which means the spray only occurred after the data collection date), we impute the most recent spray to be 3650 days (10 years ago).

In [180]:
trainSprayCombined[trainSprayCombined['Most_Recent_Spray']<3650]['Most_Recent_Spray'].describe()

count    103.000000
mean     201.300971
std      296.758396
min        1.000000
25%       15.000000
50%       35.000000
75%      642.500000
max      750.000000
Name: Most_Recent_Spray, dtype: float64

In [181]:
trainSprayCombined['Recently_Sprayed'] = (trainSprayCombined['Most_Recent_Spray']<201).astype(int)
trainSprayCombined['Recently_Sprayed'].value_counts()

0    10430
1       76
Name: Recently_Sprayed, dtype: int64

In [182]:
trainSprayCombined

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,count,max,min,ptp,Most_Recent_Spray,Recently_Sprayed
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.955,-87.801,9,1,0,NaN,NaT,NaT,NaN,3650.0,0
1,2007-05-29,CULEX RESTUANS,T002,41.955,-87.801,9,1,0,NaN,NaT,NaT,NaN,3650.0,0
2,2007-05-29,CULEX RESTUANS,T007,41.995,-87.769,9,1,0,NaN,NaT,NaT,NaN,3650.0,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974,-87.825,8,1,0,1.0,2011-09-07,2011-09-07,0.0,3650.0,0
4,2007-05-29,CULEX RESTUANS,T015,41.974,-87.825,8,4,0,1.0,2011-09-07,2011-09-07,0.0,3650.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10501,2013-09-26,CULEX PIPIENS/RESTUANS,T035,41.764,-87.742,8,6,1,NaN,NaT,NaT,NaN,3650.0,0
10502,2013-09-26,CULEX PIPIENS/RESTUANS,T231,41.987,-87.666,8,5,0,NaN,NaT,NaT,NaN,3650.0,0
10503,2013-09-26,CULEX PIPIENS/RESTUANS,T232,41.913,-87.668,9,1,0,NaN,NaT,NaT,NaN,3650.0,0
10504,2013-09-26,CULEX PIPIENS/RESTUANS,T233,42.010,-87.807,9,5,0,1.0,2013-09-05,2013-09-05,0.0,21.0,1


**Feature Engineering** to create new column 'Recently_Sprayed'. This column represents if the location has been recently sprayed, with the threshold being set to the mean of the column (disregarding the rows where 'Most_Recent_Spray'==3650).

However, based on this criteria, only a small proportion of the locations have been recently sprayed (76 out of 10430). There is very little variance within the column, which means it might not be very useful (to be assessed later in the EDA/modelling parts).

Import weather.csv file which may provide additional insights to the data.

In [183]:
weatherDf = pd.read_csv('weather.csv')
weatherDf.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [184]:
weatherDf.shape

(2944, 22)

The weatherDf contains 2944 rows and 22 columns.

In [185]:
weatherDf.isnull().sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

For now, there seems to be no null values in this dataframe. However, the data dictionary suggests that there might still be missing values, denoted by 'M' or '-' for different columns. This will be examined later.

In [186]:
weatherDf['Date'] = pd.to_datetime(weatherDf['Date'])

The weatherDf dataframe contains information from 2 different weather stations in Chicago:
- Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
- Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

Before merging this to the combined dataframe (with trainDf and sprayDf), we need to first determine which weather station to use for each row in the combined dataframe. This will be based on distance proximity - using the weather station that is closer to the collection lat/long location.

In [187]:
def calculateDistance(latDelta, longDelta):
    return np.sqrt((latDelta)**2 + (longDelta)**2)

In [188]:
def checkNearerStation(lat,long):
    station1 = {'lat':41.995, 'long':-87.933}
    station2 = {'lat':41.786, 'long':-87.752}
    dist1 = calculateDistance(lat-station1['lat'], long-station1['long'])
    dist2 = calculateDistance(lat-station2['lat'], long-station2['long'])
    if dist1 < dist2:
        station=1
    else:
        station=2
    return station

In [189]:
trainSprayCombinedW = trainSprayCombined.copy()

In [190]:
trainSprayCombinedW['Station'] = trainSprayCombinedW.apply(lambda x: checkNearerStation(x['Latitude'], x['Longitude']),axis=1)

In [191]:
trainSprayCombinedW.head()

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,count,max,min,ptp,Most_Recent_Spray,Recently_Sprayed,Station
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.955,-87.801,9,1,0,NaN,NaT,NaT,NaN,3650.0,0,1
1,2007-05-29,CULEX RESTUANS,T002,41.955,-87.801,9,1,0,NaN,NaT,NaT,NaN,3650.0,0,1
2,2007-05-29,CULEX RESTUANS,T007,41.995,-87.769,9,1,0,NaN,NaT,NaT,NaN,3650.0,0,1
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974,-87.825,8,1,0,1.0,2011-09-07,2011-09-07,0.0,3650.0,0,1
4,2007-05-29,CULEX RESTUANS,T015,41.974,-87.825,8,4,0,1.0,2011-09-07,2011-09-07,0.0,3650.0,0,1


In [192]:
trainSprayCombinedW['Station'].value_counts(normalize=True)

2    0.782696
1    0.217304
Name: Station, dtype: float64

78% of the data collection points are closer to Station 2 in terms of distance proximity.

Merge the weather information to the combined dataframe, using both 'Date' and 'Station' as merge keys.

In [193]:
trainSprayCombinedWeather = pd.merge(trainSprayCombinedW, weatherDf, 
                                     left_on=['Date','Station'], right_on=['Date','Station'], how='left')
trainSprayCombinedWeather.head()

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,count,max,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.955,-87.801,9,1,0,NaN,NaT,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
1,2007-05-29,CULEX RESTUANS,T002,41.955,-87.801,9,1,0,NaN,NaT,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
2,2007-05-29,CULEX RESTUANS,T007,41.995,-87.769,9,1,0,NaN,NaT,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974,-87.825,8,1,0,1.0,2011-09-07,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
4,2007-05-29,CULEX RESTUANS,T015,41.974,-87.825,8,4,0,1.0,2011-09-07,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5


In [194]:
trainSprayCombinedWeather.shape

(10506, 35)

In [195]:
trainSprayCombinedWeather.isnull().sum()

Date                    0
Species                 0
Trap                    0
Latitude                0
Longitude               0
AddressAccuracy         0
NumMosquitos            0
WnvPresent              0
count                9543
max                  9543
min                  9543
ptp                  9543
Most_Recent_Spray       0
Recently_Sprayed        0
Station                 0
Tmax                    0
Tmin                    0
Tavg                    0
Depart                  0
DewPoint                0
WetBulb                 0
Heat                    0
Cool                    0
Sunrise                 0
Sunset                  0
CodeSum                 0
Depth                   0
Water1                  0
SnowFall                0
PrecipTotal             0
StnPressure             0
SeaLevel                0
ResultSpeed             0
ResultDir               0
AvgSpeed                0
dtype: int64

Since we did a left merge on the trainDf, the final combined dataframe will have the same number of rows as trainDf.

In [197]:
trainSprayCombinedWeather.columns

Index(['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'count', 'max', 'min', 'ptp',
       'Most_Recent_Spray', 'Recently_Sprayed', 'Station', 'Tmax', 'Tmin',
       'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise',
       'Sunset', 'CodeSum', 'Depth', 'Water1', 'SnowFall', 'PrecipTotal',
       'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [198]:
trainSprayCombinedWeather.drop(['Station','count','max','min','ptp'], axis=1, inplace=True)

In [199]:
trainSprayCombinedWeather.isin(['M']).sum()

Date                     0
Species                  0
Trap                     0
Latitude                 0
Longitude                0
AddressAccuracy          0
NumMosquitos             0
WnvPresent               0
Most_Recent_Spray        0
Recently_Sprayed         0
Tmax                     0
Tmin                     0
Tavg                     0
Depart                8223
DewPoint                 0
WetBulb                 26
Heat                     0
Cool                     0
Sunrise                  0
Sunset                   0
CodeSum                  0
Depth                 8223
Water1               10506
SnowFall              8223
PrecipTotal             35
StnPressure             26
SeaLevel                 0
ResultSpeed              0
ResultDir                0
AvgSpeed                 0
dtype: int64

Out of 10506 rows, there are a few columns with high number of missing values ('M'):
1. Water1 (10506 - 100%)
2. Depart (8223 - 78%)
3. Depth (8223 - 78%)
4. SnowFall (8233 - 78%)

We will drop these columns since they have mostly missing values.

In [200]:
trainSprayCombinedWeather.drop(['Water1','Depart','Depth','SnowFall'], axis=1, inplace=True)

In [201]:
trainSprayCombinedWeather.isin(['-']).sum()

Date                    0
Species                 0
Trap                    0
Latitude                0
Longitude               0
AddressAccuracy         0
NumMosquitos            0
WnvPresent              0
Most_Recent_Spray       0
Recently_Sprayed        0
Tmax                    0
Tmin                    0
Tavg                    0
DewPoint                0
WetBulb                 0
Heat                    0
Cool                    0
Sunrise              8223
Sunset               8223
CodeSum                 0
PrecipTotal             0
StnPressure             0
SeaLevel                0
ResultSpeed             0
ResultDir               0
AvgSpeed                0
dtype: int64

Out of 10506 rows, there are a few columns with high number of missing values ('-'):
1. Sunrise (8223 - 78%)
2. Sunset (8223 - 78%)

We will drop these columns since they have mostly missing values.

In [202]:
trainSprayCombinedWeather.drop(['Sunrise','Sunset'], axis=1, inplace=True)

In [203]:
trainSprayCombinedWeather.columns

Index(['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Most_Recent_Spray', 'Recently_Sprayed',
       'Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'Heat', 'Cool',
       'CodeSum', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed',
       'ResultDir', 'AvgSpeed'],
      dtype='object')

In [204]:
trainSprayCombinedWeather.dtypes

Date                 datetime64[ns]
Species                      object
Trap                         object
Latitude                    float64
Longitude                   float64
AddressAccuracy               int64
NumMosquitos                  int64
WnvPresent                    int64
Most_Recent_Spray           float64
Recently_Sprayed              int64
Tmax                          int64
Tmin                          int64
Tavg                         object
DewPoint                      int64
WetBulb                      object
Heat                         object
Cool                         object
CodeSum                      object
PrecipTotal                  object
StnPressure                  object
SeaLevel                     object
ResultSpeed                 float64
ResultDir                     int64
AvgSpeed                     object
dtype: object

After dropping these columns, there are still a few columns with some missing values:
1. WetBulb (26)
2. PrecipTotal (35)
3. StnPressure(26)

For columns without missing values, convert to correct data types first.

In [205]:
trainSprayCombinedWeather['Tavg'] = trainSprayCombinedWeather['Tavg'].astype(int)
trainSprayCombinedWeather['Heat'] = trainSprayCombinedWeather['Heat'].astype(int)
trainSprayCombinedWeather['Cool'] = trainSprayCombinedWeather['Cool'].astype(int)
trainSprayCombinedWeather['SeaLevel'] = trainSprayCombinedWeather['SeaLevel'].astype(float)
trainSprayCombinedWeather['AvgSpeed'] = trainSprayCombinedWeather['AvgSpeed'].astype(float)

We can approximate missing values of WetBulb from DewPoint and temperature with this formula: Tavg-((Tavg-DewPoint)/3).

Source: http://theweatherprediction.com/habyhints/170/

In [206]:
def fillMissingWetBulb(tavg,dp,wb):
    if wb=='M':
        wb=tavg-((tavg-dp)/3)
    return wb

In [207]:
trainSprayCombinedWeather['WetBulb'] = trainSprayCombinedWeather.apply(lambda x: fillMissingWetBulb(x['Tavg'],
                                                                                                    x['DewPoint'],
                                                                                                    x['WetBulb']), axis=1)

For PrecipTotal and StnPressure, fill missing values with mode.

In [208]:
trainSprayCombinedWeather['PrecipTotal'].sort_values(ascending=True).unique()

array(['  T', '0.00', '0.01', '0.02', '0.03', '0.04', '0.06', '0.08',
       '0.09', '0.11', '0.12', '0.13', '0.14', '0.16', '0.17', '0.19',
       '0.20', '0.23', '0.24', '0.27', '0.33', '0.36', '0.39', '0.42',
       '0.44', '0.46', '0.50', '0.52', '0.58', '0.59', '0.70', '0.79',
       '0.83', '0.84', '0.87', '0.88', '0.89', '0.92', '0.95', '1.19',
       '1.31', '1.55', '3.97', 'M'], dtype=object)

Assumption: T which stands for 'trace' amounts of precipitation, should be between 0.00 and 0.01; hence, it can be converted to 0.005.

In [209]:
trainSprayCombinedWeather['PrecipTotal'] = trainSprayCombinedWeather['PrecipTotal'].apply(lambda x: 0.005 if x=='  T' else x)

In [210]:
precipMode = trainSprayCombinedWeather['PrecipTotal'].mode()[0]
precipMode

'0.00'

In [211]:
trainSprayCombinedWeather['PrecipTotal'] = trainSprayCombinedWeather['PrecipTotal'].apply(lambda x: precipMode if x=='M' 
                                                                                          else x)

In [212]:
trainSprayCombinedWeather['PrecipTotal'] = trainSprayCombinedWeather['PrecipTotal'].astype(float)

In [213]:
trainSprayCombinedWeather['StnPressure'].unique()

array(['29.39', '29.44', '29.10', '29.16', '29.45', '29.41', '29.47',
       '29.49', '29.55', '29.23', '29.28', '29.11', '29.18', '29.26',
       '29.21', '29.35', '29.29', '29.34', '29.31', '29.36', '29.06',
       '29.19', '29.17', '29.33', '29.46', '29.27', '29.40', '29.42',
       '29.24', '29.30', '28.97', '29.03', '29.15', '29.09', 'M', '29.20',
       '28.95', '28.89', '29.14', '29.08', '29.13', '29.38', '29.48',
       '29.51', '29.12', '29.25', '29.22', '29.32', '29.59', '29.65',
       '29.37', '29.01', '28.91'], dtype=object)

In [214]:
stnPressureMode = trainSprayCombinedWeather['StnPressure'].mode()[0]
stnPressureMode

'29.34'

In [215]:
trainSprayCombinedWeather['StnPressure'] = trainSprayCombinedWeather['StnPressure'].apply(lambda x: stnPressureMode if x=='M' 
                                                                                          else x)

In [216]:
trainSprayCombinedWeather['StnPressure'] = trainSprayCombinedWeather['StnPressure'].astype(float)

**Feature Engineering** to group the weather phenomena. Based on Kaggle information, hot and dry conditions are more favourable for the West Nile virus as compared to cold and wet. We can group the 'CodeSum' variables based on these 2 categories.

1. **Hot and Dry**: HZ (Haze), FU (Smoke), VA (Volcanic Ash)

2. **Cold and Wet**: +FC (Tornado), FC (Funnel Cloud), TS (Thunderstorm), GR (Hail), RA (Rain), DZ (Drizzle), SN (Snow), SG (Snow Grains), GS (Small Hail), PL (Ice Pellets), IC (Ice Crystals), FG (Fog), BR (Mist), UP (Unknown Precipitation), PY (Spray), SQ (Squall), DR (Low Drifting), SH (Shower), FZ (Freezing), MI (shallow), PR (Partial), BC (Patches), BL (Blowing), VC (Vicinity), DU (Widespread Dust), DS (Duststorm), PO (Sand Whirls), SA (Sand), SS (Sandstorm)

3. **Normal**: ' ' (Moderate)

In [217]:
trainSprayCombinedWeather['CodeSum'].unique()

array(['BR HZ', 'RA BR', 'TSRA BR HZ VCTS', ' ', 'TSRA RA BR VCTS',
       'TS TSRA RA BR HZ VCTS', 'DZ BR', 'RA BR HZ', 'HZ', 'VCTS',
       'TSRA RA DZ BR HZ', 'TSRA BR HZ', 'TSRA RA BR', 'TS TSRA RA BR',
       'BR', 'TSRA RA', 'TSRA HZ', 'TSRA', 'RA', 'TS TSRA BR', 'RA DZ BR',
       'TS TSRA BR HZ', 'TS TSRA RA VCTS', 'TSRA BR', 'TS BR', 'TS',
       'FG BR HZ', 'TS TSRA RA FG BR HZ'], dtype=object)

In [218]:
coldAndWet = ['FC','TS','GR','RA','DZ','SN','SG','GS','PL','IC','FG','BR','UP',
              'PY','SQ','DR','SH','FZ','MI','PR','BC','BL','VC','DU','DS','PO','SA','SS']
hotAndDry = ['HZ','FU','VA']

def categoriseCodeSum(codesum):
    if len(codesum)>2:
        codesum = codesum[:2]
    if codesum in coldAndWet:
        return 'coldAndWet'
    elif codesum == ' ':
        return 'normal'
    else:
        return 'hotAndDry'
    
trainSprayCombinedWeather['Weather'] = trainSprayCombinedWeather['CodeSum'].apply(lambda x: categoriseCodeSum(x))
trainSprayCombinedWeather.head()

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Most_Recent_Spray,Recently_Sprayed,...,Heat,Cool,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Weather
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.955,-87.801,9,1,0,3650.0,0,...,0,9,BR HZ,0.0,29.39,30.11,5.8,18,6.5,coldAndWet
1,2007-05-29,CULEX RESTUANS,T002,41.955,-87.801,9,1,0,3650.0,0,...,0,9,BR HZ,0.0,29.39,30.11,5.8,18,6.5,coldAndWet
2,2007-05-29,CULEX RESTUANS,T007,41.995,-87.769,9,1,0,3650.0,0,...,0,9,BR HZ,0.0,29.39,30.11,5.8,18,6.5,coldAndWet
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974,-87.825,8,1,0,3650.0,0,...,0,9,BR HZ,0.0,29.39,30.11,5.8,18,6.5,coldAndWet
4,2007-05-29,CULEX RESTUANS,T015,41.974,-87.825,8,4,0,3650.0,0,...,0,9,BR HZ,0.0,29.39,30.11,5.8,18,6.5,coldAndWet


In [219]:
trainSprayCombinedWeather['Weather'].value_counts(normalize=True)

normal        0.571197
coldAndWet    0.411384
hotAndDry     0.017419
Name: Weather, dtype: float64

**Feature Engineering** to create new columns for 'Day of Month', 'Month', 'Year', 'Day of Week'.

In [220]:
trainSprayCombinedWeather['Day_of_Month'] = trainSprayCombinedWeather['Date'].apply(lambda x: x.to_pydatetime().day)
trainSprayCombinedWeather['Month'] = trainSprayCombinedWeather['Date'].apply(lambda x: x.to_pydatetime().month)
trainSprayCombinedWeather['Year'] = trainSprayCombinedWeather['Date'].apply(lambda x: x.to_pydatetime().year)
trainSprayCombinedWeather['Day_of_Week'] = trainSprayCombinedWeather['Date'].apply(lambda x: x.to_pydatetime().weekday())

In [221]:
trainSprayCombinedWeather.head()

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Most_Recent_Spray,Recently_Sprayed,...,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Weather,Day_of_Month,Month,Year,Day_of_Week
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.955,-87.801,9,1,0,3650.0,0,...,29.39,30.11,5.8,18,6.5,coldAndWet,29,5,2007,1
1,2007-05-29,CULEX RESTUANS,T002,41.955,-87.801,9,1,0,3650.0,0,...,29.39,30.11,5.8,18,6.5,coldAndWet,29,5,2007,1
2,2007-05-29,CULEX RESTUANS,T007,41.995,-87.769,9,1,0,3650.0,0,...,29.39,30.11,5.8,18,6.5,coldAndWet,29,5,2007,1
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974,-87.825,8,1,0,3650.0,0,...,29.39,30.11,5.8,18,6.5,coldAndWet,29,5,2007,1
4,2007-05-29,CULEX RESTUANS,T015,41.974,-87.825,8,4,0,3650.0,0,...,29.39,30.11,5.8,18,6.5,coldAndWet,29,5,2007,1


In [222]:
trainSprayCombinedWeather.to_csv('trainSprayCombinedWeather.csv', index=False)

**Repeat cleaning and feature engineering for test dataset**

In [223]:
testDf = pd.read_csv('test.csv')
testDf.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [224]:
testDf.shape

(116293, 11)

In [225]:
def cleanData(df):
    df.drop(['Address','Block','Street','AddressNumberAndStreet'],axis=1,inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Latitude'] = df['Latitude'].round(3)
    df['Longitude'] = df['Longitude'].round(3)
    df = pd.merge(df,spray1,on=['Latitude','Longitude'],how='left')
    print(df[df['ptp']>0])
    df['Date'] = pd.to_datetime(df['Date'])
    df['Most_Recent_Spray'] = df['Date']-df['min']
    df['Most_Recent_Spray'] = df['Most_Recent_Spray'].dt.days
    df['Most_Recent_Spray'] = df['Most_Recent_Spray'].apply(lambda x: x if x > 0 else 3650)
    df['Recently_Sprayed'] = (df['Most_Recent_Spray']<201).astype(int)
    df['Station'] = df.apply(lambda x: checkNearerStation(x['Latitude'], x['Longitude']),axis=1)
    df = pd.merge(df, weatherDf, left_on=['Date','Station'], right_on=['Date','Station'], how='left')
    df.drop(['Station','count','max','min','ptp','Trap'], axis=1, inplace=True)
    df.drop(['Water1','Depart','Depth','SnowFall'], axis=1, inplace=True)
    df.drop(['Sunrise','Sunset'], axis=1, inplace=True)
    df['Tavg'] = df['Tavg'].astype(int)
    df['Heat'] = df['Heat'].astype(int)
    df['Cool'] = df['Cool'].astype(int)
    df['SeaLevel'] = df['SeaLevel'].astype(float)
    df['AvgSpeed'] = df['AvgSpeed'].astype(float)
    df['WetBulb'] = df.apply(lambda x: fillMissingWetBulb(x['Tavg'], x['DewPoint'], x['WetBulb']), axis=1)
    df['PrecipTotal'] = df['PrecipTotal'].apply(lambda x: 0.005 if x=='  T' else x)
    precipMode = df['PrecipTotal'].mode()[0]
    df['PrecipTotal'] = df['PrecipTotal'].apply(lambda x: precipMode if x=='M' else x)
    df['PrecipTotal'] = df['PrecipTotal'].astype(float)
    stnPressureMode = df['StnPressure'].mode()[0]
    df['StnPressure'] = df['StnPressure'].apply(lambda x: stnPressureMode if x=='M' else x)
    df['StnPressure'] = df['StnPressure'].astype(float)
    df['Weather'] = df['CodeSum'].apply(lambda x: categoriseCodeSum(x))
    df['Day_of_Month'] = df['Date'].apply(lambda x: x.to_pydatetime().day)
    df['Month'] = df['Date'].apply(lambda x: x.to_pydatetime().month)
    df['Year'] = df['Date'].apply(lambda x: x.to_pydatetime().year)
    df['Day_of_Week'] = df['Date'].apply(lambda x: x.to_pydatetime().weekday())
    return df

In [226]:
testDf = cleanData(testDf)

Empty DataFrame
Columns: [Id, Date, Species, Trap, Latitude, Longitude, AddressAccuracy, count, max, min, ptp]
Index: []


In [227]:
testDf.head()

,Id,Date,Species,Latitude,Longitude,AddressAccuracy,Most_Recent_Spray,Recently_Sprayed,Tmax,Tmin,...,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Weather,Day_of_Month,Month,Year,Day_of_Week
0,1,2008-06-11,CULEX PIPIENS/RESTUANS,41.955,-87.801,9,3650.0,0,86,61,...,29.28,29.99,8.9,18,10.0,normal,11,6,2008,2
1,2,2008-06-11,CULEX RESTUANS,41.955,-87.801,9,3650.0,0,86,61,...,29.28,29.99,8.9,18,10.0,normal,11,6,2008,2
2,3,2008-06-11,CULEX PIPIENS,41.955,-87.801,9,3650.0,0,86,61,...,29.28,29.99,8.9,18,10.0,normal,11,6,2008,2
3,4,2008-06-11,CULEX SALINARIUS,41.955,-87.801,9,3650.0,0,86,61,...,29.28,29.99,8.9,18,10.0,normal,11,6,2008,2
4,5,2008-06-11,CULEX TERRITANS,41.955,-87.801,9,3650.0,0,86,61,...,29.28,29.99,8.9,18,10.0,normal,11,6,2008,2


In [228]:
testDf.shape

(116293, 27)

In [229]:
testDf.to_csv('testDf.csv', index=False)

## Explatory Data Analysis and Cleaning

## Final Analysis and Conclusion